In [14]:
from tqdm.notebook import tqdm as progress
import pandas as pd
import plotly.express as px
from plotly.offline import plot
import dimcli
from dimcli.utils import *
import os, sys, time, json
from IPython.display import Image
from IPython.core.display import HTML 
import csv
import ast


In [2]:
#Log into Dimensions
#API key is: 9F8D648F0D7E437CB1736BEBDF007F02
#!pip install dimcli -U --quiet 

print("==\nLogging in..")
# https://digital-science.github.io/dimcli/getting-started.html#authentication
ENDPOINT = "https://app.dimensions.ai"
if 'google.colab' in sys.modules:
    import getpass
    KEY = getpass.getpass(prompt='API Key: ')  
    dimcli.login(key=KEY, endpoint=ENDPOINT)
else:
    KEY = "9F8D648F0D7E437CB1736BEBDF007F02"
    dimcli.login(key=KEY, endpoint=ENDPOINT)
dsl = dimcli.Dsl()

==
Logging in..
Dimcli - Dimensions API Client (v1.2)
Connected to: <https://app.dimensions.ai/api/dsl> - DSL v2.9
Method: manual login


In [3]:
def read_names_from_csv(file_path, encoding='utf-8'):
    all_names = []

    with open(file_path, 'r', encoding=encoding, errors='ignore') as csv_file:
        csv_reader = csv.reader(csv_file)
        
        # Assuming the first column contains first names and the second column contains last names
        for row in csv_reader:
            try:
                if len(row) >= 2:  # Ensure the row has at least two columns
                    first_names = row[1]
                    last_names = row[3]
                    full_name = first_names + " " + last_names
                    all_names.append(full_name)
            except  UnicodeDecodeError as e:
                print(f"Error decoding line {csv_reader.line_num}: {e}")

    return all_names

# Example usage
file_path = r'\Users\ashle\OneDrive\School\Capstone\names\original_names.csv'
all_names = read_names_from_csv(file_path)
all_names = all_names[1:]

# Now you can use the lists first_names and last_names as needed
print(f'ALL NAMES({len(all_names)}) :')
print("======================================================")
print(all_names)


ALL NAMES(4216) :
['Thomas James', 'Simon Karpatkin', 'Philip Majerus', 'Ralph Nachman', 'Yale Nemerson', 'Andrew Wallace', 'Chester Alper', 'Vincent Andriole', 'Stanley Appel', 'Elia Ayoub', 'Fritz Bach', 'Kurt Bloch', 'Paul Bornstein', 'Lewis Braverman', 'Bayard Clarkson', 'Herbert Cooper', 'John Dietschy', 'Chester Edelmann', 'John Ensinck', 'Yale Enson', 'Harvey Feigenbaum', 'Daniel Foster', 'Emil Freireich', 'Thomas Gaffney', 'Mehran Goulian', 'Arthur Grollman', 'Paul Gulyassy', 'Donald Harrison', 'Peter Holt', 'Richard Hong', 'Richard Hyde', 'Norman Javitt', 'Richard Johnson', 'Manuel Kaplan', 'Nicholas Kefalides', 'Hermes Kontos', 'Claude Lenfant', 'Gilbert Levinson', 'Ruven Levitan', 'Harvey Marver', 'Hugh McDevitt', 'James McGuigan', 'William Mellman', 'Charles Mengel', 'Thomas Merigan', 'Ralph Morris', 'Beverley Murphy', 'Harold Neu', 'C. Osterland', 'Robert Palmer', 'Frank Parker', 'Claude Reed', 'Morris Reichlin', 'Jack Remington', 'Jesse Roth', 'Stuart Schlossman', 'Leslie

In [4]:
def researcherIds(name):
    res = dsl.query_iterative(f"""search researchers for "{name}" return researchers""")
    listDict = res.json['researchers']
    allIds = set()
    for idv in listDict:
        allIds.add(idv['id'])

    allIds = list(allIds)  # Convert set back to a list if needed
    
    if len(allIds) > 512:    
        allIds = allIds[0:512]
    print(f'Total of {len(allIds)} unique ids for {name} ')

    # Constructing the DSL query with the list of IDs
    query_ids = ', '.join([f'"{id_val}"' for id_val in allIds])
    query = f'search researchers where id in [{query_ids}] return researchers[id+obsolete+redirect]'

    # Execute the DSL query
    res2 = dsl.query_iterative(query)
    listDict2 = res2.json['researchers']
    allWorkingIDs = set()
    for person in listDict2:
        if person['obsolete'] == 0:  # Current Working Id(s)
            allWorkingIDs.add(person["id"])
        else:
            for ids in person['redirect']:
                allWorkingIDs.add(ids)
    
    print(f'++++++++ FINAL {len(allWorkingIDs)} Working ID(s) +++++++++++')  
    print(allWorkingIDs)
    
    return allWorkingIDs
testIDs = researcherIds("Leslie Schoenfield")

Starting iteration with limit=1000 skip=0 ...
0-18 / 18 (0.70s)
===
Records extracted: 18
Starting iteration with limit=1000 skip=0 ...


Total of 18 unique ids for Leslie Schoenfield 


0-18 / 18 (2.30s)
===
Records extracted: 18


++++++++ FINAL 2 Working ID(s) +++++++++++
{'ur.0102031123.62', 'ur.014424615212.32'}


In [5]:
def allIds(namesList):
    finalNames = {}
    secondNames = {}
    zeroNames = []
    count = 0

    for name in namesList:
        try:
            validIds = researcherIds(name)

            if len(validIds) == 0:
                zeroNames.append(name)
                pass
            elif len(validIds) == 1:
                count += 1
                print(name, count)
                finalNames[name] = validIds
            elif len(validIds) == 2:
                secondNames[name] = validIds

        except Exception as e:
            print(f"An error occurred for {name}: {e}")
    return finalNames, secondNames, zeroNames
        
#resultAll, pairsAll, zeroNames = allIds(all_names)

In [ ]:
#993 one id 
print(len(resultAll))
print(len(pairsAll))
print(len(zeroNames))

def write_set_to_csv(file_path, my_set):
    with open(file_path, 'w', newline='') as csv_file:
        writer = csv.writer(csv_file)

        # Write the header
        writer.writerow(['Name', 'ID'])

        # Write the data
        for name, id_set in my_set.items():                
            id_set = list(id_set)
            writer.writerow([name, id_set]) 
            
#write_set_to_csv("nameOneIDs.csv", resultAll)
# 993
# 639
# 0

In [20]:
#open the pairs document and take the first 300, so I can do checkPairs in batches
pairsRead = {}
file_path = r'\Users\ashle\OneDrive\School\Capstone\dimensionsAI\nameTwoIDs.csv'
with open(file_path, 'r') as csv_file:
    # Create a CSV DictReader object
    csv_reader = csv.DictReader(csv_file)

    # Iterate through the rows in the CSV file
    for row in csv_reader:
        # Each row is now a dictionary with column names as keys
        pairsRead[row['Name']] = ast.literal_eval(row['ID'])
        
        
first_half_items = list(pairsRead.items())[:len(pairsRead)//2]
# Create a new dictionary using the first half items
firstHalfDictionary = dict(first_half_items)

print(len(firstHalfDictionary))
print(firstHalfDictionary)



{'Ralph Nachman': ['ur.01325114047.30', 'ur.051560516.63'], 'Chester Alper': ['ur.016156155204.55', 'ur.01147513307.66'], 'Vincent Andriole': ['ur.01051115260.37', 'ur.050142316.56'], 'Lewis Braverman': ['ur.01142160401.68', 'ur.015770660737.69'], 'Bayard Clarkson': ['ur.012417566442.78', 'ur.016124256517.45'], 'Chester Edelmann': ['ur.011437663432.90', 'ur.070214146.86'], 'Emil Freireich': ['ur.01121770341.39', 'ur.07615016432.96'], 'William Mellman': ['ur.012674010753.72', 'ur.076325405.40'], 'C. Osterland': ['ur.0113545363.34', 'ur.075231143.62'], 'Stuart Schlossman': ['ur.011234671244.43', 'ur.01003457676.34'], 'Leslie Schoenfield': ['ur.014424615212.32', 'ur.0102031123.62'], "Bert O'Malley": ['ur.01155043307.43', 'ur.07525220635.94'], 'Edward Burka': ['ur.060122524.09', 'ur.0107055733.46'], 'Hartwig Cleve': ['ur.010526472215.88', 'ur.01014074245.64'], 'John Crawhall': ['ur.015274215364.83', 'ur.075132316.20'], 'Philip Fialkow': ['ur.0740352673.91', 'ur.012624032314.31'], 'Lauran H

In [22]:
def checkPairs(ids):
    finished = {}
    for name, id_set in ids.items():
        allIds = list(id_set)  # Convert set back to a list if needed
        # Constructing the DSL query with the list of IDs
        query_ids = ', '.join([f'"{id_val}"' for id_val in allIds])
        query = f'search researchers where id in [{query_ids}] return researchers[first_name+last_name+id+research_orgs+total_grants+total_publications]'

        # Execute the DSL query
        res2 = dsl.query_iterative(query)
        listDict2 = res2.json['researchers']
        if len(listDict2) == 2:
            dict1 = listDict2[0]
            dict2 = listDict2[1]
            if 'research_orgs' in dict1.keys() and 'research_orgs' in dict2.keys():
                if len(dict1['research_orgs']) > len(dict2['research_orgs']):
                    finished[name] = dict1
                else:
                    finished[name] = dict2
            elif 'research_orgs' in dict2.keys():
                finished[name] = dict2
            else:
                finished[name] = dict1
            
        elif len(listDict2) == 1:
            finished[name] = listDict2[0]           
        print(len(finished))
        print(finished)
    return finished   
        
newPairs = checkPairs(firstHalfDictionary)
print(newPairs)


Starting iteration with limit=1000 skip=0 ...
0-2 / 2 (0.80s)
===
Records extracted: 2
Starting iteration with limit=1000 skip=0 ...


1
{'Ralph Nachman': {'first_name': 'Ralph Louis', 'id': 'ur.01325114047.30', 'last_name': 'Nachman', 'research_orgs': [{'city_name': 'Philadelphia', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.265008.9', 'latitude': 39.947906, 'linkout': ['http://www.jefferson.edu/'], 'longitude': -75.15827, 'name': 'Thomas Jefferson University', 'state_name': 'Pennsylvania', 'types': ['Education']}, {'city_name': 'New York', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.134907.8', 'latitude': 40.7625, 'linkout': ['http://www.rockefeller.edu/'], 'longitude': -73.95556, 'name': 'Rockefeller University', 'state_name': 'New York', 'types': ['Education']}, {'acronym': 'CU', 'city_name': 'Ithaca', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.5386.8', 'latitude': 42.44851, 'linkout': ['http://www.cornell.edu/'], 'longitude': -76.47862, 'name': 'Cornell University', 'state_name': 'New York', 'types': ['Education']}, {'acronym': 'YU', 'city_name

0-2 / 2 (0.90s)
===
Records extracted: 2
Starting iteration with limit=1000 skip=0 ...


2
{'Ralph Nachman': {'first_name': 'Ralph Louis', 'id': 'ur.01325114047.30', 'last_name': 'Nachman', 'research_orgs': [{'city_name': 'Philadelphia', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.265008.9', 'latitude': 39.947906, 'linkout': ['http://www.jefferson.edu/'], 'longitude': -75.15827, 'name': 'Thomas Jefferson University', 'state_name': 'Pennsylvania', 'types': ['Education']}, {'city_name': 'New York', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.134907.8', 'latitude': 40.7625, 'linkout': ['http://www.rockefeller.edu/'], 'longitude': -73.95556, 'name': 'Rockefeller University', 'state_name': 'New York', 'types': ['Education']}, {'acronym': 'CU', 'city_name': 'Ithaca', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.5386.8', 'latitude': 42.44851, 'linkout': ['http://www.cornell.edu/'], 'longitude': -76.47862, 'name': 'Cornell University', 'state_name': 'New York', 'types': ['Education']}, {'acronym': 'YU', 'city_name

0-2 / 2 (4.53s)
===
Records extracted: 2
Starting iteration with limit=1000 skip=0 ...


3
{'Ralph Nachman': {'first_name': 'Ralph Louis', 'id': 'ur.01325114047.30', 'last_name': 'Nachman', 'research_orgs': [{'city_name': 'Philadelphia', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.265008.9', 'latitude': 39.947906, 'linkout': ['http://www.jefferson.edu/'], 'longitude': -75.15827, 'name': 'Thomas Jefferson University', 'state_name': 'Pennsylvania', 'types': ['Education']}, {'city_name': 'New York', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.134907.8', 'latitude': 40.7625, 'linkout': ['http://www.rockefeller.edu/'], 'longitude': -73.95556, 'name': 'Rockefeller University', 'state_name': 'New York', 'types': ['Education']}, {'acronym': 'CU', 'city_name': 'Ithaca', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.5386.8', 'latitude': 42.44851, 'linkout': ['http://www.cornell.edu/'], 'longitude': -76.47862, 'name': 'Cornell University', 'state_name': 'New York', 'types': ['Education']}, {'acronym': 'YU', 'city_name

0-1 / 1 (0.70s)
===
Records extracted: 1
Starting iteration with limit=1000 skip=0 ...


4
{'Ralph Nachman': {'first_name': 'Ralph Louis', 'id': 'ur.01325114047.30', 'last_name': 'Nachman', 'research_orgs': [{'city_name': 'Philadelphia', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.265008.9', 'latitude': 39.947906, 'linkout': ['http://www.jefferson.edu/'], 'longitude': -75.15827, 'name': 'Thomas Jefferson University', 'state_name': 'Pennsylvania', 'types': ['Education']}, {'city_name': 'New York', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.134907.8', 'latitude': 40.7625, 'linkout': ['http://www.rockefeller.edu/'], 'longitude': -73.95556, 'name': 'Rockefeller University', 'state_name': 'New York', 'types': ['Education']}, {'acronym': 'CU', 'city_name': 'Ithaca', 'country_code': 'US', 'country_name': 'United States', 'id': 'grid.5386.8', 'latitude': 42.44851, 'linkout': ['http://www.cornell.edu/'], 'longitude': -76.47862, 'name': 'Cornell University', 'state_name': 'New York', 'types': ['Education']}, {'acronym': 'YU', 'city_name

KeyboardInterrupt: 